In [2]:
%%capture
#Surpress ugly output.

#Install datasets from huggingface.
#!pip install datasets
import torch     # PyTorch
import datasets  # Huggingface Datasets; actual data

import random

#Architecture
from torch import nn as nn

#Plot
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class Generator(nn.Module):
    def __init__(self, nl, hs, ks, stride, pad):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Sequential(
                nn.ConvTranspose2d(100, hs*(2**(nl-3)), kernel_size=ks, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(hs*(2**(nl-3))),
                nn.ReLU()
            ))
        for i in range(2, nl-1):
            self.layers.append(nn.Sequential(
                nn.ConvTranspose2d((int)((hs*(2**(nl-1)))/(2**i)), #in dimension
                                   (int)((hs*(2**(nl-1)))/(2*2**i)), #out dimension
                                   kernel_size=ks, stride=stride, padding=pad, bias=False),
                nn.BatchNorm2d((int)((hs*(2**(nl-1)))/(2*2**i))), #out dimension
                nn.ReLU()
            ))
        #output layer
        self.layers.append(nn.Sequential(
                nn.ConvTranspose2d(hs, 3, kernel_size=ks, stride=stride, padding=pad, bias=False),
                nn.Tanh()
            ))
        
    #forward pass, calls the class init
    def forward(self, x):
        for layer in self.layers:
            print('layer:', layer)
            print('before shape:', x.shape)
            x = layer(x)
            print('after shape', x.shape, '/n')
        return x

In [5]:
generator = torch.load('./best_model_gen.pt', map_location=torch.device('cpu'))

In [ ]:
#!pip install gradio
import gradio as gr
from torchvision.utils import save_image

def generate_image(seed):
    #torch.manual_seed(seed)
    noise = torch.randn(1, 100, 1, 1, device=device) #initial random noise
    generated = generator(noise)
    gem = np.rot90(np.swapaxes((generated[0].to('cpu').detach().numpy()), 0, 2), 3)
    return gem

gradio_demo = gr.Interface(
    fn = generate_image,
    inputs = [
        gr.Slider(0,999,label='Seed', default=0),
    ],
    outputs = "image",
    title="Waifu generator",
    description="Anime face generator using GANs",
)

Run the next line of code to create a share link for the gradio demo

In [ ]:
gradio_demo.launch(share=True)

Run the next line of code to NOT create a share link for the gradio demo

In [ ]:
gradio_demo.launch()